# 基础知识

## NVIDIA
### CUDA

CUDA是啥


```
查看显存命令，也可以查看cuda版本
nvidia-smi
```
## AMD



# 在线GPU环境

## 阿里云PAI

每天发送500份

### PAI-DSW

执行下面命令，然后就可以访问端口了

export GRADIO_ROOT_PATH=/${JUPYTER_NAME}/proxy/7860/

浏览器打开

https://dsw-gateway-cn-shanghai.data.aliyun.com/dsw-362554/proxy/7860/

## 魔搭社区

注册后有免费的100小时的GPU资源


# qwen 微调法律大模型

## 准备环境

```
# 下载qwen和模型
git clone https://github.com/QwenLM/Qwen.git
git clone https://www.modelscope.cn/qwen/Qwen-1_8B-Chat.git

# 安装依赖
cd Qwen
pip install -r requirements.txt

# 安装torch
方法1
下载地址：http://download.pytorch.org/whl/torch_stable.html
wget http://download.pytorch.org/whl/cu121/torch/torch-2.1.1+cu121-cp310-cp310-linux_x86_64.whl
pip install torch-2.1.1+cu121-cp310-cp310-linux_x86_64.whl

方法2
去这个网站上找好对应版本的指令：https://pytorch.org/get-started/previous-versions/
pip install torch==2.1.1+cu121 torchvision==0.15.2+cu121 torchaudio==2.1.1+cu121 torchtext==0.15.2 torchdata==0.6.10 --extra-index-url https://download.pytorch.org/whl/cu121


# 安装deepspeed
pip install "peft<0.8.0" deepspeed
```

## 微调

准备模型：Qwen-1_8B-Chat

准备训练数据:trans_data.json

```
cd Qwen

# 移动finetune_lora_single_gpu.sh到外面
cp finetune/finetune_lora_single_gpu.sh ./
chmod +x finetune_lora_single_gpu.sh

# 修改微调配置
vim finetune_lora_single_gpu.sh


# 微调，根据需要修改
./finetune_lora_single_gpu.sh -m /mnt/workspace/Qwen/Qwen-1_8B-Chat -d /mnt/workspace/data/trans_data.json

# 合并模型，模型合并文件 qwen_lora_merge.py在钉钉的文件夹里，上传到Qwen文件夹里，然后修改微调模型路径
lora_model_path="/mnt/workspace/Qwen/output_qwen/checkpoint-30"

# 合并模型
python qwen_lora_merge.py 
```

## 测试

### 交互式Demo
```
cd Qwen
python cli_demo.py -c Qwen-1_8B-Chat_merge
```

```
# 启动服务
python web_demo.py --cpu-only -c=Qwen-1_8B-Chat
# 查看web界面
http://127.0.0.1:8000/
```

## 常见问题

1、ImportError: cannot import name 'packaging' from 'pkg_resources' 
```
ImportError: cannot import name 'packaging' from 'pkg_resources' (/ibex/user/xxxx/mambaforge/envs/trivol/lib/python3.9/site-packages/pkg_resources/__init__.py)

用pip list | grep setuptools 查了一下，steuptools是 70.0.0版本。卸载重装为69.5.1版本即解决问题。

解决方案：
pip uninstall setuptools
pip install setuptools==69.5.1 --no-cache-dir
```

2、ValueError: Your setup doesn't support bf16/gpu. You need torch>=1.10, using Ampere GPU with cuda>=11.0
```
解决方案：
需要修改finetune_lora_single_gpu.sh文件，最下面有说明
--bf16 True  修改为 --fp16 True --deepspeed finetune/ds_config_zero2.json
```

3、TypeError: isin() received an invalid combination of arguments - got 
```
解决方案：
pip install coqui-tts
```

# qwen2 微调大模型

# LlamaFactory微调大模型

## 安装

```
git clone https://github.com/hiyouga/LLaMA-Factory.git

# 安装torch
# wget http://download.pytorch.org/whl/cu121/torch-2.1.1%2Bcu121-cp310-cp310-linux_x86_64.whl
# pip install torch-2.1.1+cu121-cp310-cp310-linux_x86_64.whl

cd LLaMA-Factory

pip install -e.[torch,metrics]
pip install -r requirements.txt
```
## 上传数据集

上传到LLaMA-Factory/data/目录，并修改同目录下的dataset_info.json，修改如下：

```
  "mydata": {
    "file_name": "trans_data.json",
    "formatting":"sharegpt",
    "columns": {
        "messages": "conversations",
        "tools":"id"
    },
    "tags": {
        "role_tag": "from",
        "content_tag": "value",
        "user_tag": "user",
        "assistant_tag": "assistant"
    }
  },
```

## 微调-web

pip install tf-keras

export GRADIO_ROOT_PATH=/${JUPYTER_NAME}/proxy/7860/

https://dsw-gateway-cn-shanghai.data.aliyun.com/dsw-362554/proxy/7860/

python src/webui.py

功能包括：微调train、测试chat、导出模型export

## 部署

同qwen的部署方式

## 常见问题

```
1、/bin/sh: 1: llamafactory-cli: not found
解决方案：
运行以下代码，然后重新安装依赖
pip install -e.[metrics]
pip install -r requirements.txt
```